In [ ]:
!pip list 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation"

/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation


In [ ]:
!pip install nemo_toolkit['all']
!pip install hydra-core==1.1

In [ ]:
!pip install import-ipynb

In [ ]:
import torch
import torch.nn as nn
import os

In [ ]:
from omegaconf import DictConfig, OmegaConf, open_dict
from nemo.collections.asr.data import audio_to_text_dataset
from nemo.collections.asr.parts.submodules.conformer_modules import ConformerLayer
from nemo.collections.asr.parts.utils.activations import Swish
from nemo.core.classes.mixins import AccessMixin
from nemo.core.classes.mixins.adapter_mixins import AdapterModuleMixin
from nemo.collections.asr.parts.submodules.multi_head_attention import (
    MultiHeadAttention,
    RelPositionMultiHeadAttention,
)

In [ ]:
__all__ = ['OverwrittenConformerLayer']

In [ ]:
class OverwrittenConformerLayer(ConformerLayer, AdapterModuleMixin, AccessMixin):
      def __init__(self, *args, **kwargs):
        super().__init__(
          d_model = kwargs.get("d_model"),
          d_ff = kwargs.get("d_ff"),
          self_attention_model='rel_pos',
          n_heads=8,
          conv_kernel_size=31,
          conv_norm_type='batch_norm',
          dropout=0.1,
          dropout_att=0.1,
          pos_bias_u=None,
          pos_bias_v=None,
        )
      
      def forward(self, x, att_mask=None, pos_emb=None, pad_mask=None):
        residual = x
        x = self.norm_feed_forward1(x)
        x = self.feed_forward1(x)
        residual = residual + self.dropout(x) * self.fc_factor

        x = self.norm_self_att(residual)
        if self.self_attention_model == 'rel_pos':
            x = self.self_attn(query=x, key=x, value=x, mask=att_mask, pos_emb=pos_emb)
        elif self.self_attention_model == 'abs_pos':
            x = self.self_attn(query=x, key=x, value=x, mask=att_mask)
        else:
            x = None
        self_attention_output = x
        residual = residual + self.dropout(x)
  
        x = self.norm_conv(residual)
        x = self.conv(x, pad_mask)
        residual = residual + self.dropout(x)

        x = self.norm_feed_forward2(residual)
        x = self.feed_forward2(x)
        residual = residual + self.dropout(x) * self.fc_factor

        x = self.norm_out(residual)

        if self.is_adapter_available():
            # Call the adapters
            x = self.forward_enabled_adapters(x)

        if self.is_access_enabled():
            self.register_accessible_tensor(tensor=x)

        return x, self_attention_output

In [ ]:
# x = torch.randn((8, 80, 8))
# pos_emb = torch.ones((8, 1, 8))
# overwritten_layer_obj = OverwrittenConformerLayer(d_model=8, d_ff=32)
# print(overwritten_layer_obj(x=x, pos_emb=pos_emb)[1])

tensor([[[ 2.8546e-01,  6.7624e-02, -5.9895e-03,  ...,  9.7207e-02,
          -4.4695e-01, -6.0885e-01],
         [ 2.8598e-01,  5.2135e-02, -4.8142e-02,  ...,  5.2699e-02,
          -3.9155e-01, -6.0264e-01],
         [ 3.1728e-01,  3.7103e-02, -4.3645e-02,  ...,  1.2181e-01,
          -4.6073e-01, -6.0950e-01],
         ...,
         [ 3.0120e-01,  3.8946e-02, -1.0253e-02,  ...,  2.2608e-01,
          -3.0078e-01, -6.6375e-01],
         [ 3.4858e-01,  1.1193e-02, -9.8371e-02,  ...,  1.1453e-01,
          -5.0039e-01, -6.4531e-01],
         [ 3.2043e-01,  6.5274e-02, -8.1081e-02,  ...,  4.0567e-02,
          -4.5299e-01, -5.9010e-01]],

        [[ 3.5767e-01,  9.5073e-03, -1.3106e-01,  ...,  9.8532e-02,
          -4.7123e-01, -6.2675e-01],
         [ 3.3842e-01,  4.5082e-02, -1.0532e-01,  ...,  1.4695e-01,
          -2.6257e-01, -6.5456e-01],
         [ 3.5811e-01, -1.2916e-02, -1.5105e-01,  ...,  1.0668e-01,
          -4.2706e-01, -6.0841e-01],
         ...,
         [ 3.3388e-01, -2